In [1]:
from selenium import webdriver
driver=webdriver.Firefox()
driver.implicitly_wait(3)
driver.get("https://www.fundsexplorer.com.br/ranking")

In [5]:
from selenium.webdriver.common.by import By
fecha_aceitar = '//*[@id="hs-eu-confirmation-button"]'#"/html/body/div[2]/div/div[2]/div/a"
driver.find_element(By.XPATH,fecha_aceitar).click()

In [7]:
#-------------# Muda o contexto para o iframe 
iframe_element = driver.find_element(By.XPATH, "//iframe[@title='Popup CTA']")
driver.switch_to.frame(iframe_element)
# Encontra o botão "X" dentro do iframe
close_button = driver.find_element(By.XPATH, "/html/body/div/div[1]")
# Clique no botão "X"
close_button.click()
# Volte para o contexto principal
driver.switch_to.default_content()
#------------------------------------


In [8]:
# Scroll up the window by a specific number of pixels
# For example, scroll up by 200 pixels
scroll_distance = 500
driver.execute_script(f"window.scrollBy(0, {scroll_distance});")

In [9]:
driver.find_element(By.XPATH,'//*[@id="colunas-ranking__select-button"]').click()
driver.find_element(By.XPATH,'/html/body/div[7]/div[1]/div/div[2]/div[2]/ul/li[1]/label').click()

In [10]:
dados = []
dadosTabela = driver.find_element(By.XPATH,'//div/table[contains(@class,"default-fiis-table__container__table")]')

#print(dadosTabela.text)  

for linha in dadosTabela.find_elements(By.TAG_NAME,"tr") :
    linhaDados = []
    for coluna in linha.find_elements(By.TAG_NAME,"td"):
        # print(coluna)
        linhaDados.append(coluna.text)
    dados.append(linhaDados)


In [12]:
import pandas as pd
colunas = ['Fundos','Setor','Preço Atual (R$)','Liquidez Diária (R$)',#
    'P/VP','Último Dividendo','Dividend Yield','DY (3M) Acumulado',#
    'DY (6M) Acumulado','DY (12M) Acumulado','DY (3M) média','DY (6M) média',#
    'DY (12M) média','DY Ano','Variação Preço','Rentab. Período','Rentab. Acumulada',#
    'Patrimônio Líquido','VPA','P/VPA','DY Patrimonial','Variação Patrimonial',#
    'Rentab. Patr. Período','Rentab. Patr. Acumulada','Quant. Ativos' ,'Volatilidade' , #
    'Num. Cotistas' , 'Tax. Gestão' , 'Tax. Performance' , 'Tax. Administração']

df = pd.DataFrame(dados,columns=colunas)

df

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,DY Patrimonial,Variação Patrimonial,Rentab. Patr. Período,Rentab. Patr. Acumulada,Quant. Ativos,Volatilidade,Num. Cotistas,Tax. Gestão,Tax. Performance,Tax. Administração
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,AAGR11,INDEFINIDO,"99,90","4.907,50",N/A,"1,17","1,19 %","3,46 %","6,85 %","6,85 %",...,N/A,N/A,N/A,N/A,0,"57,68",0,N/A,N/A,N/A
2,AAZQ11,INDEFINIDO,"7,81","799.879,38","0,90","0,10","1,21 %","3,96 %","7,82 %","15,69 %",...,"1,27 %","0,00 %","0,00 %","0,00 %",0,"17,53",29.430,N/A,N/A,N/A
3,ABCP11,SHOPPINGS,"70,03","48.748,32","0,69","0,62","0,93 %","2,12 %","4,07 %","9,49 %",...,"0,34 %","11,00 %","11,38 %","19,31 %",1,"15,93",15.924,N/A,N/A,N/A
4,AFHI11,PAPÉIS,"96,82","1.107.416,00","1,01","0,95","0,99 %","3,00 %","5,98 %","12,35 %",...,"0,99 %","1,41 %","2,41 %","8,83 %",0,"6,39",40.791,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,YUFI11B,INDEFINIDO,N/A,N/A,N/A,"0,63","0,00 %","0,00 %","0,00 %","0,00 %",...,"0,60 %","-0,01 %","0,59 %","6,32 %",0,N/A,0,N/A,N/A,N/A
486,ZAGH11,SERVIÇOS FINANCEIROS DIVERSOS,"12,00","878,47","1,23","0,05","0,42 %","1,35 %","1,83 %","1,83 %",...,N/A,N/A,N/A,N/A,0,"30,24",240,N/A,N/A,N/A
487,ZAVC11,INDEFINIDO,"10,49","57,13","1,05","0,11","1,11 %","3,13 %","3,13 %","3,13 %",...,N/A,N/A,N/A,N/A,0,"15,01",122,N/A,N/A,N/A
488,ZAVI11,INDEFINIDO,"115,06","186.944,94","0,87","1,10","1,09 %","3,02 %","6,03 %","11,82 %",...,N/A,N/A,N/A,N/A,10,"33,75",4.300,N/A,N/A,N/A


In [13]:
import os
data_path = str(os.getcwd()) + r"/data/"

In [15]:
df.to_csv( data_path+"FE.csv" , sep=";" , decimal="," )

In [16]:
driver.close()